In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib

pd.set_option('display.max_columns', None)


In [11]:
## fantrax rosters

fx = r"C:\Users\mikej\Desktop\fantrax\fx_wk8.csv"

teams = pd.read_csv(fx ,usecols=['Player','Status','Roster Status'])
teams.columns = ['player','team','active_reserve']
teams_dict = {'919':'Philly919','u_c':'unit_circle','NT 8':'Sneads Foot','NT 4':'New Team 4','txms':'txmoonshine','MG':'Team Gamble','grrr':'Putt Pirates','[AW]':'AlphaWired'}
teams['team'] = teams.team.map(teams_dict)
teams = teams.loc[teams.active_reserve=='Active'].set_index('player')

In [12]:
## live scoring feed

file = "https://feeds.datagolf.com/preds/live-tournament-stats?stats=sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_bs,sg_total,distance,accuracy,gir,prox_fw,prox_rgh,scrambling&round=event_avg&display=value&file_format=csv&key=e297e933c3ad47d71ec1626c299e"

live = round(pd.read_csv(file),2).rename(columns={'player_name':'player'})#.set_index('player')

names = live['player'].str.split(expand=True)
names[0] = names[0].str.rstrip(",")
names[1] = names[1].str.rstrip(",")
names['player'] = names[1] + " " + names[0]
names['player'] = np.where(names['player']=='Min Lee', 'Min Woo Lee', names['player'])
names['player'] = np.where(names['player']=='Byeong An', 'Byeong Hun An', names['player'])
names['player'] = np.where(names['player']=='Rooyen Van', 'Erik Van Rooyen', names['player'])
live = live.set_index(names.player)

In [13]:
# live look at all 48 active players with player as index

live_merged = pd.merge(teams, live, how='left', left_index=True, right_index=True)[['team','position','total','round','thru']].fillna(0).sort_values('total')

live_merged[['total','round','thru']] = live_merged[['total','round','thru']].astype('int')


live_merged['holes_remaining'] = (72 - (live_merged['thru'] + 54).fillna(0))
live_merged['holes_remaining'] = np.where(live_merged['position']=='CUT',0,live_merged['holes_remaining']).astype('int')

live_merged = live_merged.rename(columns={'position':'Pos','total':'Total','round':'Round','thru':'Thru'})

,team,Pos,Total,Round,Thru,holes_remaining
player,,,,,,
Erik Van Rooyen,New Team 4,2,-14,-8,18,0
Jake Knapp,unit_circle,3,-13,-5,15,3
Shane Lowry,unit_circle,T4,-12,1,5,13
Billy Horschel,Putt Pirates,T4,-12,-5,17,1
Keith Mitchell,Putt Pirates,T4,-12,-6,18,0
Cameron Young,New Team 4,T8,-11,-3,16,2
Sam Ryder,Sneads Foot,T16,-10,-5,18,0
Matthieu Pavon,Team Gamble,T16,-10,-2,15,3
Rickie Fowler,Sneads Foot,T16,-10,-2,10,8


In [43]:
# table showing thruCut and holes_remaining

df_cut = pd.DataFrame(live_merged[live_merged.Pos!='CUT']['team'].value_counts()).rename(columns={'team':'thruCut'})

df_holes_remaining = live_merged.groupby('team')[['holes_remaining']].sum().sort_values(by='holes_remaining',ascending=False)

pd.merge(df_cut,df_holes_remaining, left_index=True, right_index=True).T

,Sneads Foot,txmoonshine,unit_circle,Team Gamble,New Team 4,Putt Pirates,Philly919,AlphaWired
thruCut,6,6,4,4,3,3,3,3
holes_remaining,14,18,27,3,2,1,9,30


In [ ]:
team_color={
                "Philly919": 'rgb(127,60,141)',
                "unit_cirle": 'rgb(17,165,121)',
                "AlphaWired": 'rgb(57,105,172)',
                "Sneads Foot": 'rgb(242,183,1)',
                "New Team 4": 'rgb(231,63,116)',
                "Team Gamble": 'rgb(230,131,16)',
                "txmoonshine": 'rgb(0,134,139)',
                "Putt Pirates": 'rgb(165,170,153)'}

In [9]:
def row_style(row):
    if row.team == 'unit_circle':
        return pd.Series('background-color: 'rgb(17,165,121)', row.index)
    elseif: row.team == 'Philly919':
        return pd.Series('background-color: 'rgb(127,60,141)', row.index)
    else:
        return pd.Series('background-color: 'rgb(57,105,172)', row.index)

SyntaxError: unterminated string literal (detected at line 3) (1507082030.py, line 3)

In [ ]:
print(f"unit_circle has {live_merged[live_merged.team=='unit_circle']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='unit_circle'].drop(columns=['holes_remaining','team'])

In [ ]:
# live look at all 48 active players with player as index

print(f"Philly919 has {live_merged[live_merged.team=='Philly919']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='Philly919'].drop(columns=['holes_remaining','team'])

In [ ]:
# live look at all 48 active players with player as index

print(f"Team Gamble has {live_merged[live_merged.team=='Team Gamble']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='Team Gamble'].drop(columns=['holes_remaining','team'])

In [ ]:
# live look at all 48 active players with player as index

print(f"New Team 4 has {live_merged[live_merged.team=='New Team 4']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='New Team 4'].drop(columns=['holes_remaining','team'])

In [ ]:
live_merged.team.unique()

In [ ]:
def get_live():
    team_names = live_merged.team.unique()
    
    for team in team_names:
        print(f"{team} has {live_merged[live_merged.team=='New Team 4']['holes_remaining'].sum()} Holes Remaining")
        print(live_merged[live_merged.team=={team}].drop(columns='team'))

In [ ]:
get_live()

In [ ]:
unit_circle = teams.loc[(teams.team=='unit_circle') & (teams.active_reserve=='Active')].index

live['thru_hp'] = live['thru'] + 18

live[live.player.isin([unit_circle])][['player','total','round','thru','thru_hp']].reset_index(drop=True)

In [ ]:
active_rosters = []

for team in teams.team.unique():
    one_roster = teams.loc[(teams.team==team) & (teams.active_reserve=='Active')].index
    active_rosters.append(one_roster)

cols = teams.team.unique()
active_rosters = pd.DataFrame(active_rosters, index=cols).T
active_rosters

In [ ]:
def get_live(active_rosters, live):
    for team in active_rosters.columns:
        teams.loc[(teams.team==team) & (teams.active_reserve=='Active')].index